## Work Flow

1. Perform any cleaning, exploratory analysis, and/or visualizations to use the
provided data for this analysis.
   
2. Build a predictive model to help determine the probability that a rider will
be retained.

3. Evaluate the model.  Focus on metrics that are important for your *statistical
model*.
 
4. Identify / interpret features that are the most influential in affecting
your predictions.

5. Discuss the validity of your model. Issues such as
leakage.  For more on leakage, see [this essay on
Kaggle](https://www.kaggle.com/dansbecker/data-leakage), and this paper: [Leakage in Data
Mining: Formulation, Detection, and Avoidance](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.365.7769&rep=rep1&type=pdf).

6. Repeat 2 - 5 until you have a satisfactory model.

7. Consider business decisions that your model may indicate are appropriate.
Evaluate possible decisions with metrics that are appropriate for *decision
rules*.

## Deliverables

- Code you used to build the model.  The more repeatable, self explanatory, the
  better.

- A presentation including the following points:
  - How did you compute the target?
  - What model did you use in the end? Why?
  - Alternative models you considered? Why are they not good enough?
  - What performance metric did you use to evaluate the *model*? Why?
  - **Based on insights from the model, what plans do you propose to
    reduce churn?**
  - What are the potential impacts of implementing these plans or decisions?
    What performance metrics did you use to evaluate these *decisions*, why?

## Problem Description

A ride-sharing company (Company X) is interested in predicting rider retention.
To help explore this question, we have provided a sample dataset of a cohort of
users who signed up for an account in January 2014. The data was pulled on July
1, 2014; we consider a user retained if they were “active” (i.e. took a trip)
in the preceding 30 days (from the day the data was pulled). In other words, a
user is "active" if they have taken a trip since June 1, 2014. The data,
`churn.csv`, is in the [data](data) folder.  The data are split into train and
test sets.  You are encouraged to tune and estimate your model's performance on
the train set, then see how it does on the unseen data in the test set at the
end.

- The 'Feature Importance' discussion is in Random Forests lecture
- Also discussed on page 262 of hands-on

### Numerical Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from churn_eda import clean_data

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use("ggplot")

### Learning Libraries

In [31]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, roc_curve, confusion_matrix, accuracy_score, precision_score, recall_score
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression

### Load in Data

In [3]:
df = pd.read_csv('data/churn_train.csv')

train = pd.read_csv('data/churn_train.csv')

In [4]:
df = clean_data(df)

train = clean_data(train)

In [5]:
df.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,phone,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,churn,avg_rating_of_driver_nan,avg_rating_by_driver_nan,days_since_signup
0,6.94,5.0,5.000000,1.00,Astapor,Android,0.0,0,False,100.0,1,0,0,170
1,8.06,5.0,5.000000,1.00,Astapor,Android,0.0,2,True,0.0,1,0,0,157
2,21.50,4.0,4.601697,1.00,Winterfell,iPhone,0.0,1,True,100.0,1,1,0,180
3,9.46,5.0,4.601697,2.75,Winterfell,Android,100.0,1,False,100.0,1,1,0,173
4,13.77,5.0,4.601697,1.00,Winterfell,iPhone,0.0,0,False,100.0,1,1,0,151


## One Hot Encode Categorical

In [6]:
from onehotencoding import set_ohe

In [7]:
df = set_ohe(df, 'city')
df = set_ohe(df, 'phone')

In [8]:
df.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,churn,avg_rating_of_driver_nan,avg_rating_by_driver_nan,days_since_signup,city: Winterfell,city: Astapor,phone: iPhone,phone: Android
0,6.94,5.0,5.000000,1.00,0.0,0,False,100.0,1,0,0,170,0.0,1.0,0.0,1.0
1,8.06,5.0,5.000000,1.00,0.0,2,True,0.0,1,0,0,157,0.0,1.0,0.0,1.0
2,21.50,4.0,4.601697,1.00,0.0,1,True,100.0,1,1,0,180,1.0,0.0,1.0,0.0
3,9.46,5.0,4.601697,2.75,100.0,1,False,100.0,1,1,0,173,1.0,0.0,0.0,1.0
4,13.77,5.0,4.601697,1.00,0.0,0,False,100.0,1,1,0,151,1.0,0.0,1.0,0.0


## Train/Test Split

In [9]:
y = df.pop('churn').values
X = df.values

In [10]:
y

array([1, 1, 1, ..., 0, 1, 0])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

## OLS

In [69]:
# ols = sm.OLS(y_train, X_train).fit()
# ols.summary()

In [13]:
# ols_results = ols.predict(X_test)
# print('OLS MSE: ', mean_squared_error(y_test, ols_results))
# print('OLS R2: ', r2_score(y_test, ols_results))

## Logistic Regression

### Test 1: No tuning, simple run

In [24]:
lr = LogisticRegression()

In [25]:
lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:
results = lr.predict(X_test)
results

array([0, 1, 0, ..., 1, 0, 1])

In [90]:
lr_cols = np.array(df.columns).reshape(-1,1)
lr_coefs = np.round(lr.coef_, 2).reshape(-1,1)

In [96]:
lr_col_coefs = np.concatenate((lr_cols, lr_coefs), axis=1)
lr_coefs_df = pd.DataFrame(lr_col_coefs, columns=['Feature Names', 'Coefficients'])
lr_coefs_df

,Feature Names,Coefficients
0,avg_dist,0.03
1,avg_rating_by_driver,0.19
2,avg_rating_of_driver,0.05
3,avg_surge,0.18
4,surge_pct,-0
5,trips_in_first_30_days,-0.11
6,luxury_car_user,-0.9
7,weekday_pct,0
8,avg_rating_of_driver_nan,0.67
9,avg_rating_by_driver_nan,0.38


In [64]:
print(f'Logistic Regression MSE      : {mean_squared_error(y_test, results):2.5f}')
print(f'Logistic Regression R2       : {r2_score(y_test, results):2.5f}')
print(f'Logistic Regression Accuracy : {accuracy_score(y_test, results):2.5f}')
print(f'Logistic Regression Precision: {precision_score(y_test, results):2.5f}')
print(f'Logistic Regression Recall   : {recall_score(y_test, results):2.5f}')

Logistic Regression MSE      : 0.27640
Logistic Regression R2       : -0.18172
Logistic Regression Accuracy : 0.72360
Logistic Regression Precision: 0.74236
Logistic Regression Recall   : 0.85628


In [65]:
confusion_matrix(y_test, results)

array([[1868, 1863],
       [ 901, 5368]])

### Test 2: With KFolds

In [66]:
def k_folds_models(X_data, y_data, num_splits):
    kfold = KFold(n_splits=num_splits)
    
    models = []
    mse = []
    accuracies = []
    precisions = []
    recalls = []
    
    for train, validation in kfold.split(X_data):
        lr_2 = LogisticRegression()
        lr_2.fit(X_data[train], y_data[train])
        pred = lr_2.predict(X_data[validation])
        
        #save each model to go back for reference
        models.append(lr_2)
        
        #mean squared error of the y_test values and the predicted
        err = mean_squared_error(y_train[test], pred)
        mse.append(err)
        
        #accuracy scores
        accu = accuracy_score(y_train[test], pred)
        accuracies.append(accu)
        
        #precision scores
        precise = precision_score(y_train[test], pred)
        precisions.append(precise)
        
        #recall scores
        recall = recall_score(y_train[test], pred)
        recalls.append(recall)
    return (models, mse, accuracies, precisions, recalls)

In [70]:
num_splits = 3
model_3, mse_3, accuracies_3, precisions_3, recalls_3 = k_folds_models(X_train, y_train, num_splits)

print(f'Logistic Regression with KFolds: {num_splits} \n')
print(f'     Average Accuracy Score  : {np.mean(accuracies_3):2.5f}')
print(f'     Average Precision Score : {np.mean(precisions_3):2.5f}')
print(f'     Average Recall Score    : {np.mean(recalls_3):2.5f}')
print(f'     Average MSE Score       : {np.mean(mse_3):2.5f}')

Logistic Regression with KFolds: 3 

     Average Accuracy Score  : 0.61180
     Average Precision Score : 0.66342
     Average Recall Score    : 0.76382
     Average MSE Score       : 0.38820
